All you have to do is call

**heatmap = get_heatmap(model, <image as a tensor (C,H,W)>, Last Convolutional Layer, <label (positive/negative or type of cancer))**


Then call **show_heatmap(heatmap, cancer type, height of image, width of image, input img)**

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def show_images(input_img, heatmap):
  f, arr_ax = plt.subplots(2,1)
  arr_ax[0,0].plot(input_img)
  arr_ax[1,0].plot(heatmap)

def convert_to_image(tensor):
  tensor = tensor.permute(1,2,0)
  tensor = tensor * 512
  return tensor.numpy().astype(np.uint8)

def get_heatmap(model, input_tensor, target_layer, label):
  model.eval()

  gradients = []
  activations = []

  def save_gradients(module, grad_input, grad_output):
    gradients.append(grad_output[0])

  def save_activations(module, input, output):
    activations.append(output)

  input_tensor = input_tensor.unsqueeze(0).to(device)
  hook_bwd = target_layer.register_backward_hook(save_gradients)
  hook_fwd = target_layer.register_forward_hook(save_activations)
  pred = model(input_tensor)
  pred[0, label].backward()

  hook_bwd.remove()
  hook_fwd.remove()

  gradients = gradients[0]
  activations = activations[0]

  pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

  for i in range(activations.shape[1]):
    activations[:, i, :, :] *= pooled_gradients[i]

  heatmap = torch.mean(activations, dim=1).squeeze()
  heatmap = torch.relu(heatmap)
  heatmap = heatmap.detach().cpu().numpy()

  return heatmap

def normalize_heatmap(heatmap):
  heatmap = heatmap - heatmap.min()
  heatmap = heatmap / heatmap.max()
  return heatmap

def show_heatmap(heatmap, cancer_type, height, width, input_tensor):
  heatmap = normalize_heatmap(heatmap)

  heatmap = cv2.resize(
    heatmap,
    (height, width),
    interpolation=cv2.INTER_CUBIC
  )

  heatmap = cv2.applyColorMap(
    np.uint8(255 * heatmap),
    cv2.COLORMAP_JET
  )
  heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

  img = (input_tensor.squeeze(0).permute(1, 2, 0).detach().cpu().numpy())
  img = np.clip(img, 0, 1)

  alpha = 0.4
  overlay = (alpha * heatmap / 255) + ((1 - alpha) * img)
  overlay = np.clip(overlay, 0, 1)
  
  #return overlay
  plt.figure(figsize=(6,6))
  plt.imshow(overlay)
  plt.axis("off")
  plt.colorbar()
  plt.title(cancer_type)

  buf = BytesIO()
  plt.savefig(buf, format='png')
  buf.seek(0)

  img = Image.open(buf)
  img.save(save_path)

***Last Convolutional Layers***

Cervical Cancer: final_conv_layer = resnet_model.layer4[-1]

Oral Cancer: final_conv_layer = oral_cancer_model.base_model.conv_head

Colon Cancer: colon_cancer_model.base_model.conv_head

Lung Cancer: final_conv_layer = lung_cancer_model.base_model.conv_head